In [1]:
import numpy as np
import pandas as pd

## Load drivers Data

In [2]:
nascar_excel_ref = "../../data/nascar2002.xls"
drivers = pd.read_excel(nascar_excel_ref, sheet_name="drivers")
new_driver_colnames = ['driver_id', 'name', 'races', 'avg_rank', 'log_mle', 'std_error',
                       'avg_rank_1', 'lasts', 'nonlasts', 'firsts', 'nonfirsts', 'avg_rank_2', 
                       'mle', 'ar_rank', 'mle_rank', 'diff', 'log_mle_1', 'std_error_1']
drivers.columns = new_driver_colnames

## Load the races data

In [3]:
races = pd.read_excel(nascar_excel_ref, sheet_name="races", index=False)
new_races_colnames = ['driver_id','race', 'finish', 'start', 'car', 'driver', 'make',
                      'sponsor', 'pts_bonus', 'laps', 'status', 'winnings',
                      'first_place_name', 'last_place_name']
races.columns = new_races_colnames
del races.index.name

# Let's get just the required columns from the races
races_trimmed = races[['driver_id','race','finish']]

## Get a complete list (all combinations) of drivers and races

In [39]:
unq_drivers = sorted(races_trimmed["driver_id"].unique())

# Remove nan from unique drivers
unq_drivers = [x for x in unq_drivers if str(x) != 'nan']
unq_races = sorted(races_trimmed["race"].unique())
unq_races = [x for x in unq_races if str(x) != 'nan']
unq_drivers_df = pd.DataFrame(unq_drivers, columns=["driver_id"])
unq_races_df = pd.DataFrame(unq_races, columns=["race"])

# Let's cross join the columns to get all unique combinations of drivers and races
all_driver_race_combs = unq_drivers_df.assign(foo=1).merge(unq_races_df.assign(foo=1)).drop('foo', 1)
all_driver_race_combs = all_driver_race_combs.sort_values(by=['race', 'driver_id'])
all_driver_race_mg = pd.merge(all_driver_race_combs, races_trimmed,  how='left', left_on=['driver_id','race'], right_on = ['driver_id','race'])

## Functions to Derive pairwise comparisons in nascar races

In [5]:
def pawise_ind(val):
    if np.isnan(val):
        out = val
    else:
        out = (val < 0)
    return out

def get_single_race_pawise(race_num, src_dat):
    """Get all pairwise race comparisons for a single race"""
    #print(race_num)
    # Subset values for the particular race
    race_res = src_dat[src_dat["race"] == race_num]
    race_res = race_res.sort_values(by=['driver_id'])
    
    # Get pairwise differences
    arr = (race_res['finish'].values - race_res['finish'].values[:, None])
    new_race_1 = pd.concat((race_res['driver_id'], pd.DataFrame(arr, columns=race_res['driver_id'])), axis=1)
    new_race_1 = new_race_1.values[:, 1:]
    
    #vpawise_ind = np.vectorize(pawise_ind)
    #new_race_1 = vpawise_ind(new_race_1)
    # Exclude the race numbers in our finall numpy array
    return new_race_1

def get_all_pawise_comb(src_dat):
    unq_races = sorted(src_dat["race"].unique())
    all_pwise_races = [get_single_race_pawise(race_num=race, src_dat=src_dat) for race in unq_races]
    #return np.dstack(all_pwise_races)
    return all_pwise_races

## Get pairwise differences for all competitors for a single race

In [37]:
race_res = all_driver_race_mg[all_driver_race_mg["race"] == 2.0]
race_res = race_res.sort_values(by=['driver_id'])
race_res
arr = (race_res['finish'].values - race_res['finish'].values[:, None])
arr
new_race_1 = pd.concat((race_res['driver_id'], pd.DataFrame(arr, columns=race_res['driver_id'])), axis=1)
#new_race_1.shape
#new_race_1 = new_race_1.values[:, 1:]
#new_race_1
#vpawise_ind = np.vectorize(pawise_ind)
#new_race_2 = vpawise_ind(new_race_1)
#new_race_2

/Users/shamindras/anaconda/envs/sklearnprod1/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in subtract
  after removing the cwd from sys.path.


In [ ]:
test_16 = get_single_race_pawise(race_num=16.0, src_dat=all_driver_race_mg)
test_17 = get_single_race_pawise(race_num=17.0, src_dat=all_driver_race_mg)
test_16.shape, test_17.shape

## Get pairwise differences for all competitors and all races

In [ ]:
all_races_pawise = get_all_pawise_comb(src_dat=all_driver_race_mg)
len(all_races_pawise)
[print(x.shape) for x in all_races_pawise]

In [ ]:
all_races_pawise = get_all_pawise_comb(src_dat=all_driver_race_mg)

In [ ]:
all_races_pawise

In [2]:
url = "http://personal.psu.edu/drh20/code/btmatlab/nascar2002.txt"
races_trimmed = pd.read_table(url, sep=" ")

# Create a cartesian product of unique drivers and races to get every combination
unq_drivers = sorted(races_trimmed["DriverID"].unique())
unq_drivers = [x for x in unq_drivers if str(x) != 'nan']
unq_races = sorted(races_trimmed["Race"].unique())
unq_races = [x for x in unq_races if str(x) != 'nan']

# Get a dataframe 
unq_drivers_df = pd.DataFrame(unq_drivers, columns=["DriverID"])
unq_races_df = pd.DataFrame(unq_races, columns=["Race"])

# Let's cross join the columns to get all unique combinations of drivers and races
all_driver_race_combs = unq_drivers_df.assign(foo=1).merge(unq_races_df.assign(foo=1)).drop('foo', 1)
all_driver_race_combs = all_driver_race_combs.sort_values(by=['Race', 'DriverID'])
all_driver_race_mg = pd.merge(all_driver_race_combs, races_trimmed,  how='left', 
                              left_on=['DriverID','Race'], right_on = ['DriverID','Race'])

In [18]:
# Now let's do a pairwise difference in finish across drivers for a single race
# based on https://stackoverflow.com/questions/46266633/pandas-creating-difference-matrix-from-data-frame
race_num = 3.0
race_res = all_driver_race_mg[all_driver_race_mg["Race"] == race_num]
race_res = race_res.sort_values(by=['DriverID'])
race_res_val = race_res.values[:, 2]
race_res_val

array([nan,  8., 43., 12., nan, 35., 23., nan, 41., nan, nan, 16.,  7.,
       18., nan, nan, nan, 28., nan, nan, nan, nan, nan, nan, 24., nan,
       nan, nan, nan, nan,  9., 17., 33.,  2., 15., nan,  6., 10., 19.,
       nan, 36., 32., 26., 27., 25., nan, nan, 20., 30., nan,  3., 14.,
       22., nan, 34., nan, nan, nan, 40., 31., 13., 37., nan, nan, nan,
       11.,  4., nan, nan, 42., 39.,  1., nan, nan, nan, nan, 38., nan,
       29., nan, nan,  5., 21.])

In [19]:
def difference_matrix(a):
    x = np.reshape(a, (len(a), 1))
    return x - x.transpose()

In [20]:
test_diff = difference_matrix(race_res_val)
test_diff.shape

(83, 83)

In [75]:
# Now let's do a pairwise difference in finish across drivers for a single race
# based on https://stackoverflow.com/questions/46266633/pandas-creating-difference-matrix-from-data-frame
race_num = 2.0
race_res = all_driver_race_mg[all_driver_race_mg["Race"] == race_num]
race_res = race_res.sort_values(by=['DriverID'])

arr = (race_res['Place'].values - race_res['Place'].values[:, None])
new_race_1 = pd.concat((race_res['DriverID'], pd.DataFrame(arr, columns=race_res['DriverID'])), axis=1)

# Remove the first column - it has the DriverID in the pairwise matrix
new_race_1 = new_race_1.values[:, 1:]
new_race_1.shape

(166, 83)

In [63]:
all_driver_race_mg.shape
#len(unq_drivers) * len(unq_races)

(2988, 3)

In [69]:
races_trimmed

,DriverID,Race,Place
0,83,1,1
1,18,1,2
2,20,1,3
3,48,1,4
4,53,1,5
5,51,1,6
6,67,1,7
7,72,1,8
8,32,1,9
9,42,1,10
